In [1]:
from pathlib import Path
import os

In [2]:
os.chdir(Path(os.getcwd()).resolve().parents[0])

In [5]:
from src.components import get_ollama_llm, get_mongo_db_client, get_chroma_db_client

In [7]:
mongo_db_client = get_mongo_db_client()
vec_db_client = get_chroma_db_client()
from src.embedding_client import RemoteEmbedding
embed_model = RemoteEmbedding(f"http://localhost:8020")

In [9]:
query = "two men"

In [10]:
embeddings = embed_model.get_query_embedding(query)

In [13]:
len(embeddings[0])

768

In [18]:
chroma_collection = vec_db_client.get_collection('images')

In [20]:
chroma_collection.query(embeddings[0])

{'ids': [['22664',
   '24718',
   '6929',
   '11299',
   '12763',
   '22850',
   '4628',
   '24743',
   '3541',
   '12018']],
 'distances': [[1.4335427,
   1.4344275,
   1.4378197,
   1.4403863,
   1.4437007,
   1.446805,
   1.4492996,
   1.450551,
   1.4509563,
   1.4510105]],
 'embeddings': None,
 'metadatas': [[None, None, None, None, None, None, None, None, None, None]],
 'documents': [['4778593226.jpg',
   '4944016457.jpg',
   '258434606.jpg',
   '3157847991.jpg',
   '3330333217.jpg',
   '4797124142.jpg',
   '2299558545.jpg',
   '4945543678.jpg',
   '2131732576.jpg',
   '3239891140.jpg']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [22]:
from llama_index.vector_stores.chroma import ChromaVectorStore

from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import VectorStoreIndex
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.vector_stores.types import VectorStoreQuery
from llama_index.core.schema import QueryBundle

In [23]:
vec_store = ChromaVectorStore(chroma_collection = chroma_collection)

In [ ]:
from llama_index.core.vector_stores import VectorStoreQuery

In [31]:
vec_store_query = VectorStoreQuery(query_embedding = embeddings[0], similarity_top_k = 5)
results = vec_store.query(query = vec_store_query)

In [39]:
[i.text for i in results.nodes]

['4778593226.jpg',
 '4944016457.jpg',
 '258434606.jpg',
 '3157847991.jpg',
 '3330333217.jpg']

In [ ]:
index = VectorStoreIndex.from_vector_store(vec_store, embed_model=embed_model)

In [40]:
mongo_client  = get_mongo_db_client()

In [41]:
collection = mongo_client['chatbot_ui_v4']['caption']

In [42]:
docs = collection.find().limit(4)
for i in docs:
    print(i)

{'_id': ObjectId('686b569ddc158f44299de34d'), 'sent_id': '0', 'caption': 'Two young guys with shaggy hair look at their hands while hanging out in the yard.', 'filename': '1000092795.jpg', 'img_id': '0'}
{'_id': ObjectId('686b569ddc158f44299de34e'), 'sent_id': '1', 'caption': 'Two young, White males are outside near many bushes.', 'filename': '1000092795.jpg', 'img_id': '0'}
{'_id': ObjectId('686b569ddc158f44299de34f'), 'sent_id': '2', 'caption': 'Two men in green shirts are standing in a yard.', 'filename': '1000092795.jpg', 'img_id': '0'}
{'_id': ObjectId('686b569ddc158f44299de350'), 'sent_id': '3', 'caption': 'A man in a blue shirt standing in a garden.', 'filename': '1000092795.jpg', 'img_id': '0'}
